In [1]:
import random
import pandas as pd
from glob import glob
import numpy as np
import os

In [2]:
def train_validate_test_split(img_dir, class_label=0, train_percent=.7, validate_percent=.1, seed=None):
    img_list = glob(img_dir + '*.dcm')
    img_list = [os.path.join(s.split('/')[-2], s.split('/')[-1]) for s in img_list] #this is really ugly!
    df = pd.DataFrame({'image':img_list, 'label':[class_label]*len(img_list), 'split':['-']*len(img_list)})
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    df.loc[perm[:train_end], 'split'] = 'train'
    df.loc[perm[train_end:validate_end], 'split'] = 'val'
    df.loc[perm[validate_end:], 'split'] = 'test'
    return df

In [3]:
df_hr = train_validate_test_split('/project/catinous/cat_data_hardballs/hr/', class_label=0)
df_hr_cat = train_validate_test_split('/project/catinous/cat_data_hardballs/hr_cat/', class_label=1)
df_hr = df_hr.append(df_hr_cat)

df_hr.to_csv('catsinom_hr_hardballs_dataset.csv')

In [6]:
df_lr = train_validate_test_split('/project/catinous/cat_data_hardballs/lr/', class_label=0)
df_lr_cat = train_validate_test_split('/project/catinous/cat_data_hardballs/lr_cat/', class_label=1)
df_lr = df_lr.append(df_lr_cat)

df_lr.to_csv('catsinom_lr_hardballs_dataset.csv')

In [7]:
df = df_lr.append(df_hr)
df.to_csv('catsinom_combined_hardballs_dataset.csv')

In [5]:
df_lr

,image,label,split
0,lr/D4EDA423_20040121_dicom_38786.dcm,0,val
1,lr/29522EA0_20101128_dicom_8439.dcm,0,train
2,lr/AE6E3CE1_20150623_dicom_13352.dcm,0,train
3,lr/02ZIRSCG_20140826_dicom_192535.dcm,0,train
4,lr/27956E43_20100823_dicom_38760.dcm,0,train
...,...,...,...
1624,lr_cat/864A6EBF_20110202_dicom_95524.dcm,1,test
1625,lr_cat/84848F27_20141227_dicom_53712.dcm,1,test
1626,lr_cat/CAE95966_20070903_dicom_110359.dcm,1,train
1627,lr_cat/F79D9291_20110304_dicom_76391.dcm,1,train


In [3]:
df_hr = train_validate_test_split('/project/catinous/cat_data_harder/hr/', class_label=0)
df_hr_cat = train_validate_test_split('/project/catinous/cat_data_harder/hr_cat/', class_label=1)
df_hr = df_hr.append(df_hr_cat)

df_hr.to_csv('catsinom_hr_harder_dataset.csv')

In [4]:
df_lr = train_validate_test_split('/project/catinous/cat_data_harder/lr/', class_label=0)
df_lr_cat = train_validate_test_split('/project/catinous/cat_data_harder/lr_cat/', class_label=1)
df_lr = df_lr.append(df_lr_cat)

df_lr.to_csv('catsinom_lr_harder_dataset.csv')

In [5]:
df = df_lr.append(df_hr)
df.to_csv('catsinom_combined_harder_dataset.csv')